In [2]:
import json
import pandas as pd
import plotly.express as px
import os
from datetime import date, datetime, timedelta
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import argparse
import math
import pickle as pkl

C:\Users\johan\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\johan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\johan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
###########gefilterte Datei einlesen#################
#read the json file as a list of dicts
path_json="" #your path to the filtered json-file
with open(path_json) as json_file:
    data=json.load(json_file)

#convert the json-file to a dataframe
tabledf_22=pd.DataFrame.from_dict(data, orient="columns")


In [ ]:
#Schätzen für eine Tabelle

In [5]:
def preprocess_page(tabledf_page):
    #drop unneccessary columns
    tabledf_page=tabledf_page.drop(columns=['pageID', 'content', 'contentHash', 'revisionId','similarityLast',  'comment', 'position','user', 'caption'])
    
    #ValidFrom als datetime Format
    tabledf_page["validFrom"]=pd.to_datetime(tabledf_page["validFrom"], format='%Y-%m-%dT%H:%M:%SZ')
    tabledf_page["timestamp_c"]=tabledf_page.validFrom.transform(lambda x: x-x.min())
    
    #generate seperate df for each table on that page
    grouped=tabledf_page.groupby("key")
    result=[g[1]for g in list(grouped)[:]]
    
    return result

In [6]:
# fi's
def calc_fi(changerows):
    changedesc=list(reversed(changerows))
    fi=[abs(changedesc[i+1]-changedesc[i]) for i in range(len(changedesc)-1)]
    #die letzte Differenz ist das gesamte letzte Element (alle Zeilen sind neu)
    fi.append(changedesc[len(changedesc)-1])
    fi=list(fi)
    return fi

In [7]:
#n=sumi*fi
def calc_n(i, fi, changerows):
    n=sum([i*observations for i,observations in zip(i,fi)])

    #Check
  #  if n != sum(changerows):
   #     print("Something went wrong with n!", "n:",n, "sum changerows: ", sum(changerows))
    #    print("fi: ",fi)
    return n

In [8]:
#Function to calculate all required variables
def calculate_variables_N2(changerows):
    D=max(changerows)
    i=list(range(1,len(changerows)+1))
    iminus1=list(range(0,len(changerows)))
    fi=calc_fi(changerows)
    f_1=fi[0]#np.sort(changerows)[-2:][1]-np.sort(changerows)[-2:][0]
    n=calc_n(i,fi,changerows)
    C=1-(fi[0]/n)
    N1=D/C
    gamma2=max(N1*sum([i*iminus1*fi for i,iminus1,fi in zip(i,iminus1,fi)])/(n*(n-1)-1),0)
   # print("fi: ",fi, "N1: ", N1, "n: ",n)
    return f_1, gamma2, N1, D, C,n

In [9]:
#N2
def N2_estimation(C,n,gamma2, N1):
    N2= N1 +(n*(1-C))/(C)*gamma2
    if gamma2==0:
        if N2!=N1:
            print("N1 and N2 should be equal, but are not!", "N1: ",N1, "N2: ",N2)

    return N2

In [10]:
#(Chao92)
def estimate_N2_allinclusive(changerows):
    iterative_estimates=[]
    f_1_array=[]
    f_1_array.append(changerows[0])

    for e in range(1,len(changerows)):
        changerows_i=changerows[0:e+1]
        f_1, gamma2, N1, D, C,n=calculate_variables_N2(changerows_i)
        N2=N2_estimation(C,n,gamma2, N1)
        iterative_estimates.append(N2)
        f_1_array.insert(0,f_1)

    return iterative_estimates, f_1_array

In [11]:
#Single Outlier Reduction (Trushkowsky et al.)
def calc_mu_SOR(changerows, fi):
    
    #avoid dividing by zero
    if (len(changerows)-1)<1:
        den=1
    else:
        den=(len(changerows)-1)
    mu_SOR=sum([x /den  for x in fi])
    return mu_SOR

def calc_sigma_SOR(changerows, mu_SOR, fi):
    if (len(changerows)-2)<1:
        den=1
    else:
        den=(len(changerows)-2)
    sigma_SOR=math.sqrt(sum([(x-mu_SOR)**2/den for x in fi]))
    return sigma_SOR
    
def calc_f1_SOR(changerows, sigma_SOR, mu_SOR, f1):
    f1_SOR=min(f1, 2*sigma_SOR+mu_SOR)
    return f1_SOR

def calc_variables_SOR(changerows):
    fi=calc_fi(changerows)
    f1=fi[0]
    mu_SOR=calc_mu_SOR(changerows, fi)
    sigma_SOR=calc_sigma_SOR(changerows, mu_SOR, fi)
    f1_SOR=calc_f1_SOR(changerows, sigma_SOR, mu_SOR, f1)
    i=list(range(1,len(changerows)+1))
    n=calc_n(i, fi, changerows)
    D=max(changerows)
    return fi, f1, mu_SOR, sigma_SOR, f1_SOR,n, D

def calc_SOR(f1_SOR, n, D):
    N_SOR=D/(1-(f1_SOR/n))
    return N_SOR

In [12]:
#Estimation(Single Outlier Reduction (SOR))

def estimate_SOR_allinclusive(changerows):
    iterative_estimates_SOR=[]
    f_1_array_SOR=[]
    f_1_array_SOR.append(changerows[0])

    for e in range(1,len(changerows)):
        changerows_i=changerows[0:e+1]
        fi, f1, mu_SOR, sigma_SOR, f1_SOR,n, D=calc_variables_SOR(changerows_i)
        N_SOR=calc_SOR(f1_SOR, n,D)
        iterative_estimates_SOR.append(N_SOR)
        f_1_array_SOR.insert(0,f1_SOR)

    return iterative_estimates_SOR, f_1_array_SOR

In [13]:
#Convergence metric rho (Luggen et al.)

def convergence_rho(w, current, est):
    N2_eval=current.iloc[len(current)-w:][est].values

    D_eval=current.iloc[len(current)-w:]['rows']

    
    upper=sum(abs(N2_eval-D_eval)/D_eval)
    rho=upper/w
    return rho


In [14]:
#N1-UNIF (siehe Luggen et al.)
def calc_N1UNIF(changerows):
    fi=calc_fi(changerows)
    f1=fi[0]
    n=max(changerows)
    S=1-(f1/n)
    D=max(changerows)
    N1_UNIF=D/S
    
    return N1_UNIF

def estimate_N1UNIF(changerows):
    iterative_estimates_N1UNIF=[]

    for e in range(1,len(changerows)):
        changerows_i=changerows[0:e+1]
        N1_UNIF=calc_N1UNIF(changerows_i)
        iterative_estimates_N1UNIF.append(N1_UNIF)

    return iterative_estimates_N1UNIF

In [15]:
#Jack1 (siehe Luggen et al.)

def calc_jack(changerows,e):
    D=max(changerows)
    k=e
    fi=calc_fi(changerows)
    f1=fi[0]
    
    N_jack1=D+(k-1)/(k)*f1
    
    if k-1 <1:
        N_jack2=0
    else:
        N_jack2=D + (2*k-3)/(k)*f1 -((k-2)**2/(k*(k-1))*fi[1])
    
    return N_jack1, N_jack2

def estimate_Jack(changerows):
    iterative_estimates_Jack1=[]
    iterative_estimates_Jack2=[]
    
    for e in range(1,len(changerows)):
        changerows_i=changerows[0:e+1]
        N_jack1, N_jack2=calc_jack(changerows_i, e)
        iterative_estimates_Jack1.append(N_jack1)
        iterative_estimates_Jack2.append(N_jack2)
    
    return iterative_estimates_Jack1, iterative_estimates_Jack2

In [ ]:
result=preprocess_page(tabledf_22) 

In [16]:
#modified: for loop is now outside of the function

def estimate_caprecap(current):

    changerows=list(current['rows'])

    #add row numbers that start from 1 per table (are used as sample numbers for graphics)
    current['samples'] = np.arange(len(current))+1

    iterative_estimates, f_1_array=estimate_N2_allinclusive(changerows)

    #SOR
    iterative_estimates_SOR, f_1_array_SOR=estimate_SOR_allinclusive(changerows)

    #add to current dataframe
    iterative_estimates.insert(0,changerows[0])
    current['N2_allrows']=iterative_estimates
    current['f_1']=list(reversed(f_1_array))

    #Single Outlier Reduction (SOR)
    iterative_estimates_SOR.insert(0,changerows[0]) #first sample doesnt make sense to estimate
    current['N_SOR']=iterative_estimates_SOR
    current['f1_SOR']=list(reversed(f_1_array_SOR))

    #N1-UNIF (Sample Coverage and the Good-Turing Estimator)
    iterative_estimates_N1UNIF=estimate_N1UNIF(changerows)
    iterative_estimates_N1UNIF.insert(0,changerows[0]) #first sample doesnt make sense to estimate 0
    current['N1_UNIF']=iterative_estimates_N1UNIF

    #Jackknife Estimators
    iterative_estimates_Jack1, iterative_estimates_Jack2=estimate_Jack(changerows)
    iterative_estimates_Jack1.insert(0,changerows[0]) #first sample doesnt make sense to estimate
    iterative_estimates_Jack2.insert(0,changerows[0]) #first sample doesnt make sense to estimate
    current['Jack1']=iterative_estimates_Jack1
    current['Jack2']=iterative_estimates_Jack2
    
    return current


In [ ]:
#estimation for the entire page: without sample periods, alle Einträge werden in Betracht gezogen (kein rows.unique())


def estimate_species(result):

    for t in range(len(result)-1):
        current=pd.DataFrame(result[t])
        #sample1=current[(current['timestamp_c']-current['timestamp_c'].min()<=sp1)]
        #mit allen Einträgen
        changerows=list(current['rows'])#.unique()


        #add row numbers that start from 1 per table (are used as sample numbers for graphics)
        current['samples'] = np.arange(len(current))+1

        iterative_estimates, f_1_array=estimate_N2_allinclusive(changerows)

        #SOR
        iterative_estimates_SOR, f_1_array_SOR=estimate_SOR_allinclusive(changerows)

        #add to current dataframe
        iterative_estimates.insert(0,changerows[0])
        current['N2_allrows']=iterative_estimates
        current['f_1']=list(reversed(f_1_array))



        #Single Outlier Reduction (SOR)
        iterative_estimates_SOR.insert(0,changerows[0]) #first sample doesnt make sense to estimate
        current['N_SOR']=iterative_estimates_SOR
        current['f1_SOR']=list(reversed(f_1_array_SOR))


        #N1-UNIF (Sample Coverage and the Good-Turing Estimator)
        iterative_estimates_N1UNIF=estimate_N1UNIF(changerows)
        iterative_estimates_N1UNIF.insert(0,changerows[0]) #first sample doesnt make sense to estimate 0
        current['N1_UNIF']=iterative_estimates_N1UNIF


        #Jackknife Estimators
        iterative_estimates_Jack1, iterative_estimates_Jack2=estimate_Jack(changerows)
        iterative_estimates_Jack1.insert(0,changerows[0]) #first sample doesnt make sense to estimate
        iterative_estimates_Jack2.insert(0,changerows[0]) #first sample doesnt make sense to estimate
        current['Jack1']=iterative_estimates_Jack1
        current['Jack2']=iterative_estimates_Jack2




        #add current dataframe back to all the dataframes
        #braucht es das überhaupt?
        result[t]=current
    return result


In [ ]:
#estimation
result=estimate_species(result)

In [ ]:
###Evaluation: Convergence Metric#############
#variables to set the scope of tables for which the convergence metric and the graphics are done
begin=0
end=len(result)-1
w=4 #considered samples for rho#

In [1]:
##########Convergence Metric########################
estimates=[ "N2_allrows", "N_SOR", "N1_UNIF", "Jack1", "Jack2"]
estimates_rho=[ "rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1", "rho_Jack2"]

def convergence_complete(result, estimates, estimates_rho):
    
    #create dict of lists to collect the results of all estimates
    dfEvaluate=pd.DataFrame(columns=estimates_rho)

    for t in range(begin,end):#len(result)-1): #iteration over tables
        dfEvaluate=pd.DataFrame(result[t])

        estimates_rho=estimates_rho#[ "rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1", "rho_Jack2"]
        rho_dict={}

        for rest in estimates_rho:
            #set the first three entries to 0 to have the same length as the df, w takes the last 4 samples into account
           # rho_dict[rest]=[0]*(w-1) 
            rho_dict[rest]=[]##

            #dfEvaluate[rest]=""

        ##for l in range(w, len(dfEvaluate)+1): #iteration over rows in table
        for l in range(0, len(dfEvaluate)):  
            current=dfEvaluate[0:l]


            for est in range(len(estimates)): #iteration over different estimates; hier kein -1, da sonst der letzte Estimate nicht beachtet wird bei der Schätzung
                #Convergence-Fehler für alle Schätzer berechnen           
                if len(current)<w:
                    rho=np.nan
                else:
                    rho=convergence_rho(w, current, estimates[est])
                rho_dict[estimates_rho[est]].append(rho)



        #und als Spalten in df hinzufügen
        df_rho=pd.DataFrame.from_dict(rho_dict, orient='index').transpose() # in df_rho ist alles noch enthalten

        #df_rho["samples"]=dfEvaluate["samples"]

        if pd.Series(estimates_rho).isin(dfEvaluate.columns).all(): 
            #damit die Spalten nicht immer wieder hinzugefügt werden
            for est_rho in estimates_rho:
                dfEvaluate[est_rho]=df_rho[est_rho].values #.values if the indexes aren't matching (important here!!)

        else:
            #print("Rho columns are not in the dataframe.")
            df_rho.index=dfEvaluate.index
            dfEvaluate = pd.concat([dfEvaluate, df_rho], axis=1)

        result[t]=dfEvaluate
    return result
        

In [ ]:
result=convergence_complete(result)

In [ ]:
##save this to csv###
pkl.dump(result, open("Species_iterative_estimation_pkl.pkl", "wb"))

In [ ]:
###From Luggen-Paper####

def conv(estimates, distincts):
    # window size
    w = 4
    #remove infinite values
    estimates = list(filter(lambda x: x != float('Inf'), estimates))
    n = len(estimates) 
    # bail out if not enough samples
    if n < w:
        return 1 
    s = [ (abs(estimates[i] - distincts[i]) / distincts[i]) for i in range(n-w, n)] #again, mind the indices.

    return sum(s) / w


def plot(df, title, ax, idx):
    lines = ['N2_allrows', 'N_SOR', 'N1_UNIF', 'Jack1', 'Jack2','rows']

    ax.set_title(title + '\n')    

    #plot estimators
    resrow = {}
    metrics = [] 

    for idx, column in enumerate(lines):
        sns.lineplot(x=df["samples"], y=column, data=df, label=column,  ax=ax,  markers=idx, errorbar=None)

    ax.set(xlabel='Samples', ylabel='Cardinality')
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

    return df

def plotInd(df, title, ax, idx):
    # indicators
    #normieren auf [0,1]
    df['f_1_normiert'] = df['f_1']/df['rows'].max()#.astype(np.float64)
    df['Distinct'] = df['rows']/df['rows'].max()#.astype(np.float64)
    df['f1_SOR_normiert']=df['f1_SOR']/df['rows'].max()


    for idx, column in enumerate(['f_1_normiert','Distinct','f1_SOR_normiert' ]):
        sns.lineplot(x=df["samples"], y=column, data=df, label=column, ax=ax, errorbar=None)

    ax.set(xlabel='Samples', ylabel='Indicators')
    sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
    
def plotConvergence(df, title, ax, idx):
    
    lines = ["rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1"]

    #ax.set_title(title + '\n')    
    #plot estimators
    #resrow = {}
    #metrics = [] 
    
    for idx, column in enumerate(lines):
        #print("idx: ", idx, "column: ",column)
        if df[column].isnull().all():
            print("No rho for this row.")
        else:
            df = df[~np.isnan(df[column])]
            sns.lineplot(x=df["samples"], y=df[column].dropna(), data=df.dropna(), label=column,  ax=ax,  markers=idx, errorbar=None)
        #print("y-value:", df[column])

            ax.set(xlabel='Samples', ylabel='Convergence')
            sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

    #return df



In [ ]:
#################Grafiken#####################
for t in range(begin,10):#len(result)-1):

        dfEstimate = pd.DataFrame(result[t], columns=["pageTitle", "validFrom", "rows", "headings", "contentType", "key", "timestamp_c", "N2_allrows", "f_1", "samples", "f1_SOR", "N_SOR", "N1_UNIF", "Jack1", "Jack2", "rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1", "rho_Jack2"])
        f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=[8, 4])
        

        graphic_result = plot(dfEstimate, str(dfEstimate["pageTitle"].values[0]) +": " + str(dfEstimate["headings"].values[0]) + " (" + str(dfEstimate["key"].values[0]) + ")", ax1, 0) 
        plotInd(dfEstimate, str(dfEstimate["pageTitle"].values[0]) + " (" + str(dfEstimate["key"].values[0]) + ")", ax2, 0)
        plotConvergence(dfEstimate, str(dfEstimate["pageTitle"].values[0]) +": " + str(dfEstimate["headings"].values[0]) + " (" + str(dfEstimate["key"].values[0]) + ")", ax3, 0)
        
        #plt.savefig(r"\graphics\\"  +str(dfEstimate["key"].values[0]) + '.pdf')
        #logging.info( "saved " + str(estimtableate) + '.pdf' )
        #plt.close()
        plt.show()
plt.tight_layout()  
plt.show()

In [ ]:
#rho pro Tabelle gemittelt

estimates_rho=[ "rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1", "rho_Jack2"]


def calc_rho_meanpertable(begin, end, result, estimates_rho):
    columns=["key", "caption"]
    columns.extend(estimates_rho)
    print(columns)
    df_rhomean=pd.DataFrame(columns=columns)
    
    for r in range(begin,end):#len(result)-1):


        current=pd.DataFrame(result[r])

        rho_mean={}
        rho_mean["key"]=current["key"].values[0]
        rho_mean["caption"]= str(current["pageTitle"].values[0])+ ": "+ str(current["headings"].values[0])


        for rest in range(len(estimates_rho)): #ohne -1 sonst wird das letzte rho nicht beachtet
            mean=current[estimates_rho[rest]].mean()
            rho_mean[estimates_rho[rest]]=mean


        df_rhomean=df_rhomean.append(rho_mean, ignore_index=True)
    return df_rhomean

In [ ]:
###über Tabellen hinweg pro Schätzer gemittelt
def rho_mean_total(estimates_rho, df_rhomean):
    means=[]
    for mean in estimates_rho:
        m=df_rhomean[mean].mean()
        means.append(m)

    df_rhomean_mean=pd.DataFrame( columns=[ "rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1", "rho_Jack2"])
    df_rhomean_mean.loc[0]=means
    return df_rhomean_mean

In [ ]:
df_rhomean=calc_rho_meanpertable(begin, end, result, estimates_rho)
df_rhomean_mean=rho_mean_total(estimates_rho, df_rhomean)
df_rhomean_mean

In [ ]:
########Sample Periods################################

def create_sampled_tables(result_sample, period):
    if "result" in locals():
        print("result is being copied")
        result_sample=result.copy() #Kopie, damit das andere result Objekt, das nicht in samples aufgeteilt ist, weiterbesteht
    
    for table in range(len(result_sample)-1):

        df=pd.DataFrame(result_sample[table])

        dates=pd.to_datetime(df.validFrom, format='%Y-%m-%dT%H:%M:%SZ', errors="coerce") #str2time(df.validFrom).reset_index()#
        df=df.assign(date=dates)

       #this return only the date and the rows column     
       #! dfg = pd.DataFrame(df.assign(date=dates)
         #!   .groupby([pd.Grouper(key='date',  freq='3Y', origin=pd.Timestamp(min(df["validFrom"].values)))])["rows"]
           #!.max())
            #.reset_index())
    
        #this doesn't return the correct date column with the boundries of the sample periods
        dfg=df.loc[df.assign(date=dates).groupby([pd.Grouper(key='date',  freq=period, origin=pd.Timestamp(min(df["validFrom"].values)))])['rows'].agg(
    lambda x : np.nan if x.count() == 0 else x.idxmax()).dropna()]
        dfg=dfg.reset_index()
         
       # dfg=df.loc[df.assign(date=dates).groupby([pd.Grouper(key='date',  freq='3Y', origin="start")])['rows'].idxmax()].reset_index(drop=True)
        result_sample[table]=dfg
    
    return result_sample

In [ ]:
result_sample=preprocess_page(tabledf_22) 
result_sample=create_sampled_tables(result_sample, "1Y")

In [ ]:
result_sample[1]

In [ ]:
####Estimate with Sample Periods######
#estimation
result_sample=estimate_species(result_sample)

In [ ]:
####Convergence Metrics with Sample Periods######
result_sample=convergence_complete(result_sample)

In [ ]:
#####PLots#####
def plot_all(result, begin, end):
    for t in range(begin,end):#len(result)-1):

            dfEstimate = pd.DataFrame(result[t], columns=["pageTitle", "validFrom", "rows", "headings", "contentType", "key", "timestamp_c", "N2_allrows", "f_1", "samples", "f1_SOR", "N_SOR", "N1_UNIF", "Jack1", "Jack2", "rho_N2_allrows", "rho_N_SOR", "rho_N1_UNIF", "rho_Jack1", "rho_Jack2"])
            f, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=[8, 4])


            graphic_result = plot(dfEstimate, str(dfEstimate["pageTitle"].values[0]) +": " + str(dfEstimate["headings"].values[0]) + " (" + str(dfEstimate["key"].values[0]) + ")", ax1, 0) 
            plotInd(dfEstimate, str(dfEstimate["pageTitle"].values[0]) + " (" + str(dfEstimate["key"].values[0]) + ")", ax2, 0)
            plotConvergence(dfEstimate, str(dfEstimate["pageTitle"].values[0]) +": " + str(dfEstimate["headings"].values[0]) + " (" + str(dfEstimate["key"].values[0]) + ")", ax3, 0)

            plt.savefig(r"\graphics\\"  +str(dfEstimate["key"].values[0]) + '.pdf')
            #plt.savefig(args.outpath + 'figures/' +str(table) + '.png')
            #logging.info( "saved " + str(estimtableate) + '.pdf' )
            #plt.close()
            plt.show()

In [ ]:
plot_all(result_sample, 0, 10)

In [ ]:
df_rho_samples=calc_rho_meanpertable(0, len(result_sample)-1, result_sample, estimates_rho)
mean_rho_samples=rho_mean_total(estimates_rho, df_rho_samples)

In [ ]:
mean_rho_samples

In [ ]:
df_rhomean_mean